In [ ]:
code = '''
import time
import sys

huge_output = 'fdsfsfdsffsfasfd' * 100

print('dfsfdfasfs', flush = True, end = '')
time.sleep(1)
#for i in range(0, 30):
#    print(huge_output, flush = True)
print('==endline', flush = True)
time.sleep(1)
print("fatal error", file=sys.stderr, flush = True)
time.sleep(3)
print('bccasfdsfdsfds', flush = True)
#x = input('Input:')
x = 'xxx'
print(f'x={x}', flush = True)
time.sleep(1)
print('ending...', flush = True)
exit(-1)
'''

In [ ]:
import subprocess
import threading
import time
import io
import logging
import asyncio
    
class PW:
    _exe = None
    _args = None
    _proc = None
    _std_cb = None
    _err_cb = None
    _threads = []
    
    _calling_cmd = None
    
    def __init__(
        self, 
        exe, 
        args = None,
        # Throw CalledProcessError when error code != 0
        check = True
    ):
        self._exe = exe
        self._args = args
        self._check = check
    
    def add_callback(
        self,
        std = None,
        err = None
    ):
        self._std_cb = std
        self._err_cb = err
    
    def _join_threads(self):
        for t in self._threads:
            t.join()
        self._threads = []
        
    def run(self):
        if len(self._threads) != 0:
            # ids = [id(t) for t in self._threads]
            # logging.error(f'Current threads {ids} must be ended first, joining...')
            self._join_threads()

        args = []
        if self._args != None:
            args = self._args if isinstance(self._args, list) else [self._args]
        
        self._calling_cmd = [self._exe]
        self._calling_cmd.extend(args)
        
        # print(calling_cmd)
        
        self._proc = subprocess.Popen(self._calling_cmd,
                                      stdin=subprocess.PIPE,
                                      stdout=subprocess.PIPE,
                                      stderr=subprocess.PIPE)
                
        def stdout_reader(proc, s):
#             from functools import partial

#             for line in iter(partial(proc.stdout.read, 1), b''):
#                 if s._std_cb != None:
#                     s._std_cb(s, line.decode('utf-8'))
            
            reader = io.TextIOWrapper(proc.stdout, encoding='utf8')
            while buf := reader.readline():
                if s._std_cb != None:
                    s._std_cb(s, buf)

#             while buf := reader.read(1):
#                 if s._std_cb != None:
#                     s._std_cb(s, buf)
#                 if buf == '\\n':
#                     print()
#                 else:
#                     print(buf, end='')
#             while something:
#                 char = reader.read(1)
#                 print(char)
                
        def stderr_reader(proc, s):
            reader = io.TextIOWrapper(proc.stderr, encoding='utf8')
            while buf := reader.readline():
                if s._err_cb != None:
                    s._err_cb(s, buf)

        self._threads.append(
            threading.Thread(target = stdout_reader, args=(self._proc, self))
        )
        
        self._threads.append(
            threading.Thread(target = stderr_reader, args=(self._proc, self))
        )
        
        for t in self._threads:
            t.start()

    def wait(self):
        logging.info("Start waiting for process to finish")

        self._join_threads()
        
        try:
            self._proc.stdout.close()
            self._err_code = self._proc.wait()
            self._proc = None
            if self._err_code != 0 and self._check:
                raise subprocess.CalledProcessError(self._err_code, self._calling_cmd)
        finally:
            if self._err_code:
                # only print error log when the CalledProcessError is not raised
                if not self._check:
                    logging.error('Process ended with error code {0}'.format(self._err_code))
            else:
                logging.info('Process ended successfully.')

In [ ]:
import sys

#PW(exe = 'ls', args = ['-a', '-l']).run()
#sys.executable, '-c', code
pw = PW(exe = sys.executable, args = ['-c', code], check = False)
pw.add_callback(
    std = lambda o,l: print('STDOUT:' + l, end=''),
    err = lambda o,l: print('STDERR:' + l, end='')
)
pw.run()
pw.wait()